In [1]:
%%javascript
//FOR MAKING EVERYTHING TABS IN JUYPYTER NOTEBOOK, LEAVE HERE FOR NOW
//ALSO MAKE SURE YOU *ALWAYS* RUN THIS WHENEVER YOU OPEN THE NOTEBOOK
//BEFORE EVERY DOING ANYTHING
IPython.tab_as_tab_everywhere = function(use_tabs) {
    if (use_tabs === undefined) {
        use_tabs = true; 
    }

    // apply setting to all current CodeMirror instances
    IPython.notebook.get_cells().map(
        function(c) {  return c.code_mirror.options.indentWithTabs=use_tabs;  }
    );
    // make sure new CodeMirror instances created in the future also use this setting
    CodeMirror.defaults.indentWithTabs=use_tabs;

    };

IPython.tab_as_tab_everywhere()

<IPython.core.display.Javascript object>

In [208]:
max_iter = 125

sample_size = 20000

#the ratio of sentences containing tags we care about/dont care about
care_ratio = 0.3

task3_care = 30000
task3_dont_care = 5000

task4_care = 30000
task4_dont_care = 5000

#turn this off for handin, whether the model is verbose or not
verbose_on = True

In [3]:
import sys, codecs, json, math, time, warnings
warnings.simplefilter( action='ignore', category=FutureWarning )

import nltk, scipy, sklearn, sklearn_crfsuite, sklearn_crfsuite.metrics, eli5
nltk.download('averaged_perceptron_tagger')
from sklearn.metrics import make_scorer
from collections import Counter
import matplotlib.pyplot as plt
from IPython.display import display    
import numpy as np

import logging
import tensorflow as tf
import absl.logging
formatter = logging.Formatter('[%(levelname)s|%(filename)s:%(lineno)s %(asctime)s] %(message)s')
absl.logging.get_absl_handler().setFormatter(formatter)
absl.logging._warn_preinit_stderr = False
logger = tf.get_logger()
logger.setLevel(logging.INFO)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/george/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [4]:
#Takes the whole book that has been read as a string and generates
#the dictionary needed containing the table of contents of the book
def task1(book_str):
    
	contents = r"(CONTENTS|(C|c)ontents)"
	
	#either a roman numeral or digit
	number = r"([ivxlcdm]+|[IVXLCDM]+|\d+)"
	
	#the charcters which have been seen to appear in titles I've looked at
	chset = r"[A-Za-z0-9\?\-\.\'\":“”’]"

In [5]:
#Takes the chapter that has been read as a string and extracts all 
#questions from it into the required set
def task2(chapter_str):
    
    #Anything can be part of a question
    #starts with a capital letter followed by any number of 
    # "non-ending" characters
	question_regex = r"[A-Z][^\?\.,!]*\?"
    
    #doesnt really need to be match objects, maybe change this later
	return set([match_obj[0] for match_obj in re.finditer(question_regex, chapter_str, flags = re.MULTILINE | re.DOTALL)])

In [6]:
#Tasks 3 and 4 are sort of linked so MAYBE handling all their initialisation
#stuff make more sense within a single function? Not sure, leave here for now

#def init_NER(chapter_str):

In [7]:
#Takes the chapter that has been read as a string and finds the 
#NER entitites listed in the task 3 spec, generating a dictionary
#containing these that the names found in task 4 can then be 
#injected into
#def task3(chapter_str):
    

In [8]:
#Takes the chapter that has been read as a string and finds the names
#of all characters, returning a list that can be put into the task 3/4
#dictionary writer
#def task4(chapter_str):
    

In [9]:
#SUBTASK 1, takes the whole book file as an argument

#all you have to do is populate dictTOC as shown commented out and 
#the function will handle writing everything for you etc.
def exec_regex_toc( file_book = None ) :

	book_str = ""
	with open(file_book, 'r') as f:
		book_str = f.read()
    
	dictTOC = task1(book_str)
        
        
	# INSERT CODE TO USE REGEX TO BUILD A TABLE OF CONTENTS FOR A BOOK (subtask 1)

#	# hardcoded output to show exactly what is expected to be serialized
#	dictTOC = {
#			"1": "I AM BORN",
#			"2": "I OBSERVE",
#			"3": "I HAVE A CHANGE"
#		}

	return dictTOC

	# DO NOT CHANGE THE BELOW CODE WHICH WILL SERIALIZE THE ANSWERS FOR THE AUTOMATED TEST HARNESS TO LOAD AND MARK

	"""
	writeHandle = codecs.open( 'toc.json', 'w', 'utf-8', errors = 'replace' )
	strJSON = json.dumps( dictTOC, indent=2 )
	writeHandle.write( strJSON + '\n' )
	writeHandle.close()
	"""

In [10]:
#SUBTASK 2, take the file chapter as argument and handles writing the list of questions

#all you ahve to do is populate setQuestions as is shown commented out
#and the function will then handle writing everything and stuff for you
def exec_regex_questions( file_chapter = None ) :

	chapter_str = ""
	with open(file_chapter, 'r') as f:
		chapter_str = f.read()
    
	setQuestions = task2(chapter_str)
    
	# INSERT CODE TO USE REGEX TO LIST ALL QUESTIONS IN THE CHAPTER OF TEXT (subtask 2)

	# hardcoded output to show exactly what is expected to be serialized
#	setQuestions = set([
#		"Traddles?",
#		"And another shilling or so in biscuits, and another in fruit, eh?",
#		"Perhaps you’d like to spend a couple of shillings or so, in a bottle of currant wine by and by, up in the bedroom?",
#		"Has that fellow’--to the man with the wooden leg--‘been here again?"
#		])

	# DO NOT CHANGE THE BELOW CODE WHICH WILL SERIALIZE THE ANSWERS FOR THE AUTOMATED TEST HARNESS TO LOAD AND MARK

	return setQuestions
	
	"""
	writeHandle = codecs.open( 'questions.txt', 'w', 'utf-8', errors = 'replace' )
	for strQuestion in setQuestions :
		writeHandle.write( strQuestion + '\n' )
	writeHandle.close()
	"""

In [11]:
from sklearn.model_selection import ShuffleSplit
from nltk.corpus import names
from nltk import pos_tag

In [12]:
#take the file path of ontonotes and return sentences
def load_ontonotes(dataset_file):

	readHandle = codecs.open(dataset_file, 'r', 'utf-8', errors = 'replace')
	str_json = readHandle.read()
	readHandle.close()
	dict_ontonotes = json.loads(str_json)
	orig_list = list(dict_ontonotes.keys())
	sentences = []

	for str_file in orig_list:
		for str_sent_index in dict_ontonotes[str_file] :

			tokens = []
			ner_tags = []
			# compute IOB tags for named entities (if any)
			ne_type_last = None

			#build up the list of tokens and ner tags
			for nTokenIndex in range(len(dict_ontonotes[str_file][str_sent_index]['tokens'])) :
				strToken = dict_ontonotes[str_file][str_sent_index]['tokens'][nTokenIndex]
				strPOS = dict_ontonotes[str_file][str_sent_index]['pos'][nTokenIndex]
				ne_type = None
				if 'ne' in dict_ontonotes[str_file][str_sent_index] :
					dict_ne = dict_ontonotes[str_file][str_sent_index]['ne']
					if not 'parse_error' in dict_ne :
						for str_NEIndex in dict_ne :
							if nTokenIndex in dict_ne[str_NEIndex]['tokens'] :
								ne_type = dict_ne[str_NEIndex]['type']
								break
				if ne_type != None :
					if ne_type == ne_type_last :
						strIOB = 'I-' + ne_type
					else :
						strIOB = 'B-' + ne_type
				else :
					strIOB = 'O'

				ne_type_last = ne_type
				tokens.append(strToken)
				ner_tags.append(strIOB)
			list_entry = []

			#use nltk pos tags instead
			for (index, tup) in enumerate(pos_tag(tokens)):
				#print("tup is:", tup)
				list_entry.append((tup[0], tup[1], ner_tags[index]))
			sentences.append(list_entry)
	return sentences

In [13]:
from sklearn.model_selection import ShuffleSplit
from nltk.corpus import names
from copy import copy
from random import shuffle
nltk.download("names")

[nltk_data] Downloading package names to /home/george/nltk_data...
[nltk_data]   Package names is already up-to-date!


True

In [211]:
def gen_NER_dict(tag_list, sentences, sentence_NER_predictions):
	
	dct = {tag : [] for tag in tag_list}
	
	for (sentence, predictions) in zip(sentences,sentence_NER_predictions):
		pairs = gen_tok_NER_pair_lists(tag_list, sentence, predictions)
		for (entity, tag) in pairs:
			dct[tag].append(entity.lower())
	
	dct = {tag : list(set(entities)) for (tag,entities) in dct.items()}
	return dct

#They should be exactly the same length as the predictions have been made from them
def gen_tok_NER_pair_lists(tag_list, sentence_with_unknown_NER, NER_predictions):
	
	
	tokens = [tok for (tok,_,_) in sentence_with_unknown_NER]
	#print("tokens:")
	#print(tokens)
	#print("predictions")
	#print(NER_predictions)
	
	pair_list = []
	
	i = 0
	while i < len(NER_predictions):

		if(NER_predictions[i] == "O"):
			i += 1
			continue
		
		#if(not (NER_predictions[i][2:] in tag_list)):
		#	i += 1
		#	continue
		
		#remove the B- or I-
		actual_tag = NER_predictions[i][2:]
		
		#The indices will correspond, start the entity with the first token with the tag
		entity = tokens[i]
		i += 1
		
		while i < len(NER_predictions) and NER_predictions[i][2:] == actual_tag:
			entity += " " + tokens[i]
			i += 1
		
		pair_list.append((entity, actual_tag))
		i += 1
	
	#print("sentence:", " ".join(tokens))
	#print("entities found:")
	#print(pair_list)
	
	return pair_list
	
	"""
	pair_lists = []
	print("unkown:")
	print(sentences_with_unknown_NER)
	
	print("predictions:")
	print(NER_predictions)
	
	for (i, sentence) in enumerate(sentences_with_unknown_NER):
		print("(testing) sentence is:")
		print(sentence)
		pair_list = []
		for thing in enumerate(sentence):
			print("(testing) thing is:")
			print(thing)
			pair_list.append((token, NER_predictions[i][j]))
		pair_lists.append(pair_list)
	
	return pair_lists
	"""

	"""
	pair_lists = []
	for i in range(len(NER_predictions)):
		pair_list = []
		for j in range(len(NER_predictions[i])):
			token = sentences_with_unknown_NER[i][j][0]
			ner_tag = NER_predictions[i][j]
			pair_list.append((token, ner_tag))
		pair_lists.append(pair_list)
	return pair_lists
	"""

In [15]:
from math import floor

task3model = []
task4model = []

#given the loaded ontonotes sentences and the sentences from the chapter
#which is having entities extracted from it, return the final NER dictionary for tasks 3 and 4
def run_NER(sentences, sentences_to_predict):

	#given a specific NER tag, return 2 sets consisting of the 
	#"inside parts" and the "before parts"
	def extract_gazetteers(target, sentences):
		before = set()
		inside = set()
		before_tag = "B-" + target
		inside_tag = "I-" + target
		for sent in sentences:
			for tok, _, ner in sent:
				if ner == before_tag:
					before.add(tok.lower())
				if ner == inside_tag:
					inside.add(tok.lower())
		return before, inside


	#convert all the NER tags you don't care about into O's in sentences that may have mixed NER tags that we care about
	#with NER tags that we don't care about
	def only_Os(desired_set, sentences):
		actual_tags = []
		for tag in desired_set:
			actual_tags.append("B-" + tag)
			actual_tags.append("I-" + tag)
		return [[(tok, pos, ner if ner in actual_tags else 'O') for tok, pos, ner in sent] for sent in sentences]


	#If the dataset has already been split and we KNOW that none of these NER tags are the ones we care about,
	#then clearly we can just replace ALL NER tags with O
	def all_Os(sentences):
		return [[(tok, pos, 'O') for tok, pos, _ in sent] for sent in sentences]


	#returns a tuples of 2 lists of sentences, where one list contains all the sentences that contain the tags 
	#specified in the argument and the other list does not
	#you just give it the tag strings and it adds the Bs and Is for you
	def split_tags(tag_set, sentences):

		actual_tags = []
		for tag in tag_set:
			actual_tags.append("B-" + tag)
			actual_tags.append("I-" + tag)

		contains_tags = []
		doesnt_contain_tags = []

		for sent in sentences:

			if(any([tag in actual_tags for _,_,tag in sent])):
				contains_tags.append(sent)
			else:
				doesnt_contain_tags.append(sent)

		return (contains_tags, doesnt_contain_tags)

	#the sentences containing task 3 tags and the sentences that dont
	wtask3, wotask3 = split_tags(["DATE", "CARDINAL", "ORDINAL", "NORP"], sentences)

	
	#the sentences containing task 4 tags (only person) and the sentences that dont
	wtask4, wotask4 = split_tags(["PERSON"], sentences)

	#all other tags can be converted to O because we dont care about finding them
	wtask3 = only_Os(["DATE", "CARDINAL", "ORDINAL", "NORP"], wtask3)
	wtask4 = only_Os(["PERSON"], wtask4)
	wotask3 = all_Os(wotask3)
	wotask4 = all_Os(wotask4)

		#take a random sample with the specified size from the list of sentences
	def random_sample(sentences, num):
		
		if num > len(sentences):
			return sentences
		
		cp = copy(sentences)
		shuffle(cp)
		return cp[-num:]
	
	#process data, putting in a ratio of sentences that contain the NER tags we care about
	#and the complement ration of ones we dont
	def process_data(care_about, dont_care):
		
		num_care = floor(care_ratio * sample_size)
		print("num_care is:", num_care)
		num_dont_care = sample_size - num_care
		print("num_dont_care is:", num_dont_care)
		
		care_sample = random_sample(care_about, num_care)
		dont_care_sample = random_sample(dont_care, num_dont_care)
		
		processed = care_sample + dont_care_sample
		shuffle(processed)
		return processed
	
	task3_processed = process_data(wtask3, wotask3)
	task4_processed = process_data(wtask4, wotask4)
	
	
	#split the items into training and testing denoted by the testing ratio argument
	def random_split(items, ratio):
		shuffle(copy(items))
		train_size = int(ratio * len(items))
		test_size = len(items) - train_size
		return items[:train_size], items[-test_size:]

	#don't need training and testing for handin, just use the whole thing
	#task3_training, task3_testing = random_split(task3_processed, 0.9)
	#task4_training, task4_testing = random_split(task4_processed, 0.9)

	#generate the gazetteers to be used from ontonotes
	#before_name, inside_name = extract_gazetteers("PERSON", wtask4)
	#before_date, inside_date = extract_gazetteers("DATE", wtask3)
	#before_cardinal, inside_cardinal = extract_gazetteers("CARDINAL", wtask3)
	#before_norp, inside_norp = extract_gazetteers("NORP", wtask3)
	#before_ord, inside_ord = extract_gazetteers("ORDINAL", wtask3)

		#assumed to be in the environment of the function that 
	#generates the feature dictionary for task 3
	number_gazetteer = set([
		"one", "two", "three", "four", "five", "six", "seven", "eight",
		"nine", "ten", "eleven", "twelve", "thirteen", "fourteen", "fifteen",
		"sixteen", "seventeen", "eighteen", "nineteen", "twenty", "thirty", "fourty", "fifty",
		"sixty", "seventy", "eighty", "ninety", "hundred", "onehundred", "one-hundred", "thousand", "million"
		])

	#Python sets are implemented as hash tables so lookup is always around O(1)
	#even for large sets

	ordinal_gazetteer = set([
		"first", "second", "third", "fourth", "fifth", "sixth", "seventh",
		"eigth", "ninth", "tenth", "eleventh", "twelth", "thirteenth",
		"fourteenth", "fifteenth", "sixteenth", "seventeenth", "eighteenth",
		"nineteenth", "twentieth", "thirtieth", "fourtieth", "fiftieth", 
		"sixtieth", "seventieth", "eightieth", "ninetieth", "hundreth", "thousandth",
		"millionth", "firstly", "secondly", "thirdly", "fourthly", "fifthly", "sixthly",
		"seventhly", "eigthly", "ninethly", "tenthly"
	])

	date_gazetteer = set([
		"monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday", "day", "week", "month", "year"
		"jan", "january", "feb", "february", "mar", "march", "apr", "april", "may", "jun", "june", "jul", "july",
		"aug", "august", "sep", "september", "oct", "october", "nov", "november", "dec", "december", "holiday", "holidays"
	])

	time_gazetteer = set(["second", "seconds", "minute", "minutes", "hour", "hours", "time", "clock", "o'clock", "past", "oclock"])

	# a feature for detecting e.g. 1st, 2nd, 3rd, 4th etc.
	def is_st_th(candidate):
		suffixes = set(["st", "nd", "rd", "th"])
		return candidate[-2:] in suffixes and candidate[:-2].isdigit()

	#use the nltk name corpus
	name_gazetteer = set(map(lambda name: name.lower(), names.words("male.txt") + names.words("female.txt")))

	#task 4 has less features as it only revolves around names
	def gen_feature_dict_task4(sentence, i):

		def gen_features(index, relative_str):
			return {
				relative_str + "word.lower()": sentence[index][0].lower(),
				relative_str + "word_prefix": sentence[index][0][:3],
				relative_str + "word_suffix": sentence[index][0][-3:],
				relative_str + "postag": sentence[index][1],
		  #relative_str + "postag_prefix2": sentence[index][1][:2], THINK THESE HARM THE MODEL
				#relative_str + "postag_prefix3": sentence[index][1][:3], #think this is useless?
				relative_str + "istitle": sentence[index][0].istitle(),
				relative_str + "isupper": sentence[index][0].isupper()     
			}

		#extra features are limited to center 3 in the context window of size 5
		def gen_extra_features(index, relative_str):
			return {
				relative_str + "contains_digit" : any(char.isdigit() for char in sentence[index][0]),
				relative_str + "in_nltk_name_gaz" :  sentence[index][0].lower() in name_gazetteer,
#				relative_str + "in_B_name" : sentence[index][0].lower() in before_name,
#				relative_str + "in_I_name" : sentence[index][0].lower() in inside_name
			}

		features = gen_features(i, "")
		features.update(gen_extra_features(i, ""))

		#at least 1 word behind
		if i > 0:#generate a list of feature dictionaries for each word in the sentence
			features.update(gen_features(i - 1, "-1:"))
			features.update(gen_extra_features(i - 1, "-1:"))

		#at least 2 words behind
		if i > 1:
			features.update(gen_features(i - 2, "-2:"))

		#at least 1 word ahead
		if i < len(sentence) - 1:
			features.update(gen_features(i + 1, "+1:"))
			features.update(gen_extra_features(i + 1, "+1:"))

		#at least 2 words ahead
		if i < len(sentence) - 2:
			features.update(gen_features(i + 2, "+2:"))

		return features

	def gen_feature_dict_task3(sentence, i):

		#token = sentence[i][0]
		#pos_tag = sentence[i][1]

		def gen_features(index, relative_str):

			return {
				relative_str + "word.lower()": sentence[index][0].lower(),
				relative_str + "word_prefix": sentence[index][0][:3],
				relative_str + "word_suffix": sentence[index][0][-3:],
				relative_str + "postag": sentence[index][1],
		  #relative_str + "postag_prefix2": sentence[index][1][:2], THINK THESE HARM THE MODEL
				#relative_str + "postag_prefix3": sentence[index][1][:3], #think this is useless?
				relative_str + "istitle": sentence[index][0].istitle(),
				relative_str + "isupper": sentence[index][0].isupper(),        
			}

		#extra features are limited to center 3 in the context window of size 5
		def gen_extra_features(index, relative_str):
			return {
				relative_str + "contains_digit": any(char.isdigit() for char in sentence[index][0]), 
				relative_str + "contains_day": "day" in sentence[index][0].lower(),

				#python sets are around O(1) lookup so its fine to have large gazetteers
				relative_str + "in_num_gaz": sentence[index][0].lower() in number_gazetteer,
				relative_str + "in_ord_gaz": sentence[index][0].lower() in ordinal_gazetteer,
				relative_str + "in_date_gaz": sentence[index][0].lower() in date_gazetteer,
				relative_str + "in_time_gaz": sentence[index][0].lower() in time_gazetteer,
				relative_str + "in_nltk_name_gaz": sentence[index][0].lower() in name_gazetteer,

				#COMMENT THESE OUT MAYBE COS NOT SURE ABOUT THEM
#				relative_str + "in_B_date": sentence[index][0].lower() in before_date,
#				relative_str + "in_I_date": sentence[index][0].lower() in inside_date,
#				relative_str + "in_B_card": sentence[index][0].lower() in before_cardinal,
#				relative_str + "in_I_card": sentence[index][0].lower() in inside_cardinal,
#				relative_str + "in_B_norp": sentence[index][0].lower() in before_norp,
#				relative_str + "in_I_norp": sentence[index][0].lower() in inside_norp,
#				relative_str + "in_B_ord": sentence[index][0].lower() in before_ord,
#				relative_str + "in_I_ord": sentence[index][0].lower() in inside_ord,
				relative_str + "is_st_th": is_st_th(sentence[index][0])
			}

		features = gen_features(i, "")
		features.update(gen_extra_features(i, ""))

		#there is at least 1 word behind
		if i > 0:
			features.update(gen_features(i - 1, "-1:"))
			features.update(gen_extra_features(i - 1, "-1:"))

		#there are at least 2 words behind
		if i > 1:
			features.update(gen_features(i - 2, "-2:"))

		#there is at least 1 word ahead
		if i < len(sentence) - 1:
			features.update(gen_features(i + 1, "+1:"))
			features.update(gen_extra_features(i + 1, "+1:"))

		#there are at least 2 words ahead
		if i < len(sentence) - 2:
			features.update(gen_features(i + 2, "+2:"))

		return features

		#helper functions
	def sent2labels(sentence):
		return [label for _, _, label in sentence] 
	def sent2tokens(sent):
		return [token for token, _, _ in sentence]

	def sentences2labels(sentences):
		return [sent2labels(sent) for sent in sentences]

	
		#generate a list of feature dictionaries for each word in the sentence
	def gen_task3_features(sentence):
		return [gen_feature_dict_task3(sentence, i) for i in range(len(sentence))]

	def gen_task4_features(sentence):
		return [gen_feature_dict_task4(sentence, i) for i in range(len(sentence))]

	display_label_subset = ["B-DATE", "I-DATE", "B-CARDINAL", "I-CARDINAL", "B-ORDINAL", "I-ORDINAL", "B-NORP", "I-NORP"]

	
	#generate lists of feature dictionaries for each sentence
	def task3(ontonotes_data, sentences_to_predict):
		
		#print("Task 3 sentences to predict length:", len(sentences_to_predict))
		
		feature_lists = [gen_task3_features(sent) for sent in ontonotes_data]
		feature_dicts_to_predict = [gen_task3_features(sent) for sent in sentences_to_predict]
		#print("The first feature dict in the first feature dict list is:")
		#print(feature_lists[0][0])
		#print("The first feature dict in the first feature dict list (the ones that should be predicted) is:")
		#print(feature_dicts_to_predict[0][0])
		
		#print("Made the task 3 feature lists")
		label_lists = [sent2labels(sent) for sent in ontonotes_data]
		crf = sklearn_crfsuite.CRF(
			algorithm = "lbfgs",
			c1 = 10,
			c2 = 0.149853957,
			max_iterations = max_iter,
			all_possible_transitions = True,
			verbose = verbose_on
		)
		crf.fit(feature_lists, label_lists)
		
		#the tuple thing means show the top 30 and also the worst 30
		print("TRANSITION WEIGHTS LEARNT ON DATASET:\n")
		display( eli5.show_weights(crf, top=(30,30), targets = display_label_subset, show=['transition_features']))
		print("\nBEST PERFORMING FEATURES FOR EACH NER TAG:\n")
		display( eli5.show_weights(crf, top=(30,30), targets = display_label_subset, show=['targets']) )
		
		#print("feature list length:")
		task3model.append(crf)
		predictions = crf.predict(feature_dicts_to_predict)
		
		#print("first thing in predictions is:")
		#print(predictions[0])
		
		#print("and the first sentence is:")
		#print(sentences_to_predict[0])
		
		dct = gen_NER_dict(["PERSON", "CARDINAL", "ORDINAL", "DATE", "NORP"], sentences_to_predict, predictions)
		print("THE DICT OF PREDICTIONS WAS:")
		print(dct)
		
		#predictions = crf.predict(feature_dicts_to_predict)
		#print("TASK 3 PREDICTIONS:")
		#print(gen_tok_NER_pair_lists(sentences_to_predict, predictions))
		return {}
		#return extract_entities_from_sequences(
		#	["CARDINAL", "ORDINAL", "DATE", "NORP"], predictions, True)

	#try seeing if different hyperparamaters for each model improves
	#their respsective performances, might be worth them not both being the same
	def task4(ontonotes_data, sentences_to_predict):
		
		#print("Task 4 sentences to predict length:", len(sentences_to_predict))
		feature_lists = [gen_task4_features(sent) for sent in ontonotes_data]
		feature_dicts_to_predict = [gen_task4_features(sent) for sent in sentences_to_predict]
		#print("The first feature dict in the first feature dict list is:")
		#print(feature_lists[0][0])
		#print("The first feature dict in the first feature dict list (the ones that should be predicted) is:")
		#print(feature_dicts_to_predict[0][0])
		label_lists = [sent2labels(sent) for sent in ontonotes_data]
		crf = sklearn_crfsuite.CRF(
			algorithm = "lbfgs",
			c1 = 10,
			c2 = 0.149853957,
			max_iterations = max_iter,
			all_possible_transitions = True,
			verbose = verbose_on
		)
		crf.fit(feature_lists, label_lists)
		task4model.append(crf)
		#predictions = crf.predict(feature_dicts_to_predict)
		#print("TASK 4 PREDICTIONS:")
		#print(gen_tok_NER_pair_lists(sentences_to_predict, predictions))
		return {}
		#return extract_entities_from_sequences(["PERSON"], predictions, False)

	print("task 3 processed size:")
	print(len(task3_processed))
	
	print("task 4 processed size:")
	print(len(task4_processed))
		
	#Think this is all that my compie can handle :)
	task3_processed = random_sample(task3_processed, sample_size)
	task4_processed = random_sample(task4_processed, sample_size)
	
	print("TASK 3 SIZE:")
	print(len(task3_processed))
	
	print("TASK 4 SIZE:")
	print(len(task4_processed))
	
	ner_dict = task3(task3_processed, sentences_to_predict)
	ner_dict.update(task4(task4_processed, sentences_to_predict))

	task3model = task3model[0]
	task4model = task4model[0]
	
	#phewwww finally :)
	return ner_dict



In [16]:
#DOOOOOO NOT RUN THIS CELl!
#(unless its the first time opening the notebook though obviously)

from nltk import sent_tokenize, word_tokenize
ontonotes_sentences = load_ontonotes("ontonotes_parsed.json")

In [204]:
#turn the list of tokens into a sentence that can be used for NER
def prepare_sentence(tokens):
	tags = list(map(lambda x: x[1], pos_tag(tokens)))
	sentence = []
	for i in range(len(tokens)):
		sentence.append((tokens[i],tags[i],"O"))
	return sentence

#detect if the chapter string has "CHAPTER . SOMETHING" at the start of it
#and return the string without it
def without_chapter(chapter_str):
	
	#Define the end of a title as there being at least 2 new line characters
	regex = r"(\n|[ ])*.*(?:CHAPTER|(C|c)hapter)[ ]*\.?(?:[ivxlcdm]+|[IVXLCDM]+|\d+)[ ]*\.?(\n|[ ])*.*\n.*\n(\n|[ ])*"
	result = re.match(regex, chapter_str, flags = re.MULTILINE | re.UNICODE)
	
	return chapter_str[len(result[0]):]
	
	#if there is a Chapter N part at the start of the chapter then remove it
	#length = len(result[0])
	#return chapter_str[length:]

In [202]:
#Used for taking the chapter file and ontonotes dataset as an argument
#then training the task 3 and task 4 models and writing the dictionary
#results for each of them

#all you have to do is fill up dictNE as is shown commented out and
#the function handles writing everything for you
def exec_ner( file_chapter = None, ontonotes_file = None ) :

	#First read the whole chapter in as a string
	chapter_str = ""
	with open(file_chapter, 'r') as f:
		chapter_str = without_chapter(f.read())
	sentences_to_predict = []
	for sent in sent_tokenize(chapter_str):
		tokens = word_tokenize(sent)
		#sentence = []
		#Start with initially all "O" tags (these are then replaced by NER predictions obviously)
		#for i in range(len(tokens)):
		#	sentence.append((tokens[i], tags[i], "O"))
		sentences_to_predict.append(prepare_sentence(tokens))
	
	#ontonotes_sentences = load_ontonotes(ontonotes_file)
	dictNE = run_NER(ontonotes_sentences, sentences_to_predict)
	
	# INSERT CODE TO TRAIN A CRF NER MODEL TO TAG THE CHAPTER OF TEXT (subtask 3)
	# USING NER MODEL AND REGEX GENERATE A SET OF BOOK CHARACTERS AND FILTERED SET OF NE TAGS (subtask 4)

	# hardcoded output to show exactly what is expected to be serialized (you should change this)
#	dictNE = {
#			"CARDINAL": [
#				"two",
#				"three",
#				"one"
#			],
#			"ORDINAL": [
#				"first"
#			],
#			"DATE": [
	#			"saturday",
#			],
#			"NORP": [
#				"indians"
#			],
#			"PERSON": [
#				"creakle",
#				"mr. creakle",
#				"mrs. creakle",
#				"miss creakle"
#			]
#		}

	return dictNE

"""
	# DO NOT CHANGE THE BELOW CODE WHICH WILL SERIALIZE THE ANSWERS FOR THE AUTOMATED TEST HARNESS TO LOAD AND MARK

	# write out all PERSON entries for character list for subtask 4
	writeHandle = codecs.open( 'characters.txt', 'w', 'utf-8', errors = 'replace' )
	if 'PERSON' in dictNE :
		for strNE in dictNE['PERSON'] :
			writeHandle.write( strNE.strip().lower()+ '\n' )
	writeHandle.close()

	# FILTER NE dict by types required for subtask 3
	listAllowedTypes = [ 'DATE', 'CARDINAL', 'ORDINAL', 'NORP' ]
	listKeys = list( dictNE.keys() )
	for strKey in listKeys :
		for nIndex in range(len(dictNE[strKey])) :
			dictNE[strKey][nIndex] = dictNE[strKey][nIndex].strip().lower()
		if not strKey in listAllowedTypes :
			del dictNE[strKey]

	# write filtered NE dict
	writeHandle = codecs.open( 'ne.json', 'w', 'utf-8', errors = 'replace' )
	strJSON = json.dumps( dictNE, indent=2 )
	writeHandle.write( strJSON + '\n' )
	writeHandle.close()
	"""

"\n\t# DO NOT CHANGE THE BELOW CODE WHICH WILL SERIALIZE THE ANSWERS FOR THE AUTOMATED TEST HARNESS TO LOAD AND MARK\n\n\t# write out all PERSON entries for character list for subtask 4\n\twriteHandle = codecs.open( 'characters.txt', 'w', 'utf-8', errors = 'replace' )\n\tif 'PERSON' in dictNE :\n\t\tfor strNE in dictNE['PERSON'] :\n\t\t\twriteHandle.write( strNE.strip().lower()+ '\n' )\n\twriteHandle.close()\n\n\t# FILTER NE dict by types required for subtask 3\n\tlistAllowedTypes = [ 'DATE', 'CARDINAL', 'ORDINAL', 'NORP' ]\n\tlistKeys = list( dictNE.keys() )\n\tfor strKey in listKeys :\n\t\tfor nIndex in range(len(dictNE[strKey])) :\n\t\t\tdictNE[strKey][nIndex] = dictNE[strKey][nIndex].strip().lower()\n\t\tif not strKey in listAllowedTypes :\n\t\t\tdel dictNE[strKey]\n\n\t# write filtered NE dict\n\twriteHandle = codecs.open( 'ne.json', 'w', 'utf-8', errors = 'replace' )\n\tstrJSON = json.dumps( dictNE, indent=2 )\n\twriteHandle.write( strJSON + '\n' )\n\twriteHandle.close()\n\t"

In [209]:
chapter_str = ""
with open("eval_chapter.txt", 'r') as f:
	chapter_str = without_chapter(f.read())

sentences_to_predict = []
for sent in sent_tokenize(chapter_str):
	tokens = word_tokenize(sent)
	#sentence = []
	#Start with initially all "O" tags (these are then replaced by NER predictions obviously)
	#for i in range(len(tokens)):
	#	sentence.append((tokens[i], tags[i], "O"))
	sentences_to_predict.append(prepare_sentence(tokens))

In [113]:
import re

In [214]:
sentences = ontonotes_sentences

task3model = []
task4model = []

#given a specific NER tag, return 2 sets consisting of the 
#"inside parts" and the "before parts"
def extract_gazetteers(target, sentences):
	before = set()
	inside = set()
	before_tag = "B-" + target
	inside_tag = "I-" + target
	for sent in sentences:
		for tok, _, ner in sent:
			if ner == before_tag:
				before.add(tok.lower())
			if ner == inside_tag:
				inside.add(tok.lower())
	return before, inside


#convert all the NER tags you don't care about into O's in sentences that may have mixed NER tags that we care about
#with NER tags that we don't care about
def only_Os(desired_set, sentences):
	actual_tags = []
	for tag in desired_set:
		actual_tags.append("B-" + tag)
		actual_tags.append("I-" + tag)
	return [[(tok, pos, ner if ner in actual_tags else 'O') for tok, pos, ner in sent] for sent in sentences]


#If the dataset has already been split and we KNOW that none of these NER tags are the ones we care about,
#then clearly we can just replace ALL NER tags with O
def all_Os(sentences):
	return [[(tok, pos, 'O') for tok, pos, _ in sent] for sent in sentences]


#returns a tuples of 2 lists of sentences, where one list contains all the sentences that contain the tags 
#specified in the argument and the other list does not
#you just give it the tag strings and it adds the Bs and Is for you
def split_tags(tag_set, sentences):

	actual_tags = []
	for tag in tag_set:
		actual_tags.append("B-" + tag)
		actual_tags.append("I-" + tag)

	contains_tags = []
	doesnt_contain_tags = []

	for sent in sentences:

		if(any([tag in actual_tags for _,_,tag in sent])):
			contains_tags.append(sent)
		else:
			doesnt_contain_tags.append(sent)

	return (contains_tags, doesnt_contain_tags)

#the sentences containing task 3 tags and the sentences that dont
wtask3, wotask3 = split_tags(["DATE", "CARDINAL", "ORDINAL", "NORP"], sentences)


#the sentences containing task 4 tags (only person) and the sentences that dont
wtask4, wotask4 = split_tags(["PERSON"], sentences)

#all other tags can be converted to O because we dont care about finding them
wtask3 = only_Os(["DATE", "CARDINAL", "ORDINAL", "NORP"], wtask3)
wtask4 = only_Os(["PERSON"], wtask4)

#Any sentences that don't contain any entities we care about can
#ALL be converted to Os!
wotask3 = all_Os(wotask3)
wotask4 = all_Os(wotask4)

	#take a random sample with the specified size from the list of sentences
def random_sample(sentences, num):

	if num > len(sentences):
		return sentences

	cp = copy(sentences)
	shuffle(cp)
	return cp[-num:]

#process data, putting in a ratio of sentences that contain the NER tags we care about
#and the complement ration of ones we dont
def process_data(care_about, dont_care):

	num_care = floor(care_ratio * sample_size)
	print("num_care is:", num_care)
	num_dont_care = sample_size - num_care
	print("num_dont_care is:", num_dont_care)

	care_sample = random_sample(care_about, num_care)
	dont_care_sample = random_sample(dont_care, num_dont_care)

	processed = care_sample + dont_care_sample
	shuffle(processed)
	return processed

#task3_processed = process_data(wtask3, wotask3)
#task4_processed = process_data(wtask4, wotask4)


#split the items into training and testing denoted by the testing ratio argument
def random_split(items, ratio):
	shuffle(copy(items))
	train_size = int(ratio * len(items))
	test_size = len(items) - train_size
	return items[:train_size], items[-test_size:]

#don't need training and testing for handin, just use the whole thing
#task3_training, task3_testing = random_split(task3_processed, 0.9)
#task4_training, task4_testing = random_split(task4_processed, 0.9)

#generate the gazetteers to be used from ontonotes
#before_name, inside_name = extract_gazetteers("PERSON", wtask4)
#before_date, inside_date = extract_gazetteers("DATE", wtask3)
#before_cardinal, inside_cardinal = extract_gazetteers("CARDINAL", wtask3)
#before_norp, inside_norp = extract_gazetteers("NORP", wtask3)
#before_ord, inside_ord = extract_gazetteers("ORDINAL", wtask3)

	#assumed to be in the environment of the function that 
#generates the feature dictionary for task 3
number_gazetteer = set([
	"one", "two", "three", "four", "five", "six", "seven", "eight",
	"nine", "ten", "eleven", "twelve", "thirteen", "fourteen", "fifteen",
	"sixteen", "seventeen", "eighteen", "nineteen", "twenty", "thirty", "fourty", "fifty",
	"sixty", "seventy", "eighty", "ninety", "hundred", "onehundred", "one-hundred", "thousand", "million"
	])

#Python sets are implemented as hash tables so lookup is always around O(1)
#even for large sets

ordinal_gazetteer = set([
	"first", "second", "third", "fourth", "fifth", "sixth", "seventh",
	"eigth", "ninth", "tenth", "eleventh", "twelth", "thirteenth",
	"fourteenth", "fifteenth", "sixteenth", "seventeenth", "eighteenth",
	"nineteenth", "twentieth", "thirtieth", "fourtieth", "fiftieth", 
	"sixtieth", "seventieth", "eightieth", "ninetieth", "hundreth", "thousandth",
	"millionth", "firstly", "secondly", "thirdly", "fourthly", "fifthly", "sixthly",
	"seventhly", "eigthly", "ninethly", "tenthly"
])

date_gazetteer = set([
	"monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday", "day", "days", 
	"week", "weeks", "month", "months", "year", "years", "decade", "decades", "century", "centuries", 
	"millennium", "millennia", "jan", "january", "feb", "february", "mar", "march", "apr", "april", 
	"may", "jun", "june", "jul", "july", "aug", "august", "sep", "september", "oct", "october", 
	"nov", "november", "dec", "december", "holiday", "holidays"
])

time_gazetteer = set(["second", "seconds", "minute", "minutes", "hour", "hours", "time", "clock", "o'clock", "past", "oclock"])

title_gazetteer = set([
	"mr", "mr.", "mrs", "mrs.", "miss", "miss.", "madam", 
	"mam.", "sir", "sir.", "lord", "lord.", "mam", "mister", 
	"mister.", "missus", "missus.", "dame", "chairman", "king", 
	"queen", "president"])

# a feature for detecting e.g. 1st, 2nd, 3rd, 4th etc.
def is_st_th(candidate):
	suffixes = set(["st", "nd", "rd", "th"])
	return candidate[-2:] in suffixes and candidate[:-2].isdigit()

#use the nltk name corpus
#Keep all names uppercase as by convention names will only ever really 
#appear in books as uppercase, this rules out things like "will" being
#detected as a name
name_gazetteer = set(names.words("male.txt") + names.words("female.txt"))

#task 4 has less features as it only revolves around names
def gen_feature_dict_task4(sentence, i):

	def gen_features(index, relative_str):
		return {
			relative_str + "word.lower()": sentence[index][0].lower(),
			relative_str + "word_prefix": sentence[index][0][:3],
			relative_str + "word_suffix": sentence[index][0][-3:],
			relative_str + "postag": sentence[index][1],
			relative_str + "is_mr_mrs": sentence[index][0][:2].lower() == "mr" or sentence[index][0][:3].lower() == "mrs",
			#relative_str + "in_title_gaz": sentence[index][0].lower() in title_gazetteer,
			relative_str + "not_alnum": not sentence[index][0].isalnum(),
			relative_str + "is_single_letter": sentence[index][0].isalpha() and len(sentence[index][0]) == 1
		}

	#extra features are limited to center 3 in the context window of size 5
	def gen_extra_features(index, relative_str):
		return {
			relative_str + "istitle": sentence[index][0].istitle(),
			relative_str + "isupper": sentence[index][0].isupper(),
			relative_str + "in_nltk_name_gaz" :  sentence[index][0] in name_gazetteer
		}

	features = gen_features(i, "")
	features.update(gen_extra_features(i, ""))

	#at least 1 word behind
	if i > 0:#generate a list of feature dictionaries for each word in the sentence
		features.update(gen_features(i - 1, "-1:"))
		features.update(gen_extra_features(i - 1, "-1:"))

	#at least 2 words behind
	if i > 1:
		features.update(gen_features(i - 2, "-2:"))

	#at least 1 word ahead
	if i < len(sentence) - 1:
		features.update(gen_features(i + 1, "+1:"))
		features.update(gen_extra_features(i + 1, "+1:"))

	#at least 2 words ahead
	if i < len(sentence) - 2:
		features.update(gen_features(i + 2, "+2:"))

	return features

def gen_feature_dict_task3(sentence, i):

	#token = sentence[i][0]
	#pos_tag = sentence[i][1]

	def gen_features(index, relative_str):

		return {
			relative_str + "word.lower()": sentence[index][0].lower(),
			relative_str + "word_prefix": sentence[index][0][:3],
			relative_str + "word_suffix": sentence[index][0][-3:],
			relative_str + "postag": sentence[index][1],
			relative_str + "isdigit": sentence[index][0].isdigit()
		}

	#extra features are limited to center 3 in the context window of size 5
	def gen_extra_features(index, relative_str):
		return {
			relative_str + "not_alnum": not sentence[index][0].isalnum(),
			relative_str + "istitle": sentence[index][0].istitle(),
			relative_str + "isupper": sentence[index][0].isupper(),
			#python sets are around O(1) lookup so its fine to have large gazetteers
			relative_str + "in_num_gaz": sentence[index][0].lower() in number_gazetteer,
			relative_str + "in_ord_gaz": sentence[index][0].lower() in ordinal_gazetteer,
			relative_str + "in_date_gaz": sentence[index][0].lower() in date_gazetteer,
			relative_str + "in_time_gaz": sentence[index][0].lower() in time_gazetteer,
			#relative_str + "in_nltk_name_gaz": sentence[index][0].lower() in name_gazetteer,

			#COMMENT THESE OUT MAYBE COS NOT SURE ABOUT THEM
#				relative_str + "in_B_date": sentence[index][0].lower() in before_date,
#				relative_str + "in_I_date": sentence[index][0].lower() in inside_date,
#				relative_str + "in_B_card": sentence[index][0].lower() in before_cardinal,
#				relative_str + "in_I_card": sentence[index][0].lower() in inside_cardinal,
#				relative_str + "in_B_norp": sentence[index][0].lower() in before_norp,
#				relative_str + "in_I_norp": sentence[index][0].lower() in inside_norp,
#				relative_str + "in_B_ord": sentence[index][0].lower() in before_ord,
#				relative_str + "in_I_ord": sentence[index][0].lower() in inside_ord,
			
			#e.g. 1st, 2nd, 3rd, 4th etc.
			relative_str + "is_st_th": is_st_th(sentence[index][0])
		}

	features = gen_features(i, "")
	features.update(gen_extra_features(i, ""))

	#there is at least 1 word behind
	if i > 0:
		features.update(gen_features(i - 1, "-1:"))
		features.update(gen_extra_features(i - 1, "-1:"))

	#there are at least 2 words behind
	if i > 1:
		features.update(gen_features(i - 2, "-2:"))

	#there are at least 3 words behind
	if i > 2:
		features.update(gen_features(i - 3, "-3:"))

	#there is at least 1 word ahead
	if i < len(sentence) - 1:
		features.update(gen_features(i + 1, "+1:"))
		features.update(gen_extra_features(i + 1, "+1:"))

	#there are at least 2 words ahead
	if i < len(sentence) - 2:
		features.update(gen_features(i + 2, "+2:"))

	#there are at least 3 words ahead
	if i < len(sentence) - 3:
		features.update(gen_features(i + 3, "+3:"))

	return features

	#helper functions
def sent2labels(sentence):
	return [label for _, _, label in sentence] 
def sent2tokens(sent):
	return [token for token, _, _ in sentence]

def sentences2labels(sentences):
	return [sent2labels(sent) for sent in sentences]


	#generate a list of feature dictionaries for each word in the sentence
def gen_task3_features(sentence):
	return [gen_feature_dict_task3(sentence, i) for i in range(len(sentence))]

def gen_task4_features(sentence):
	return [gen_feature_dict_task4(sentence, i) for i in range(len(sentence))]

display_label_subset = ["B-DATE", "I-DATE", "B-CARDINAL", "I-CARDINAL", "B-ORDINAL", "I-ORDINAL", "B-NORP", "I-NORP"]


#generate lists of feature dictionaries for each sentence
def task3(ontonotes_data, sentences_to_predict):


	feature_lists = [gen_task3_features(sent) for sent in ontonotes_data]
	feature_dicts_to_predict = [gen_task3_features(sent) for sent in sentences_to_predict]
	label_lists = [sent2labels(sent) for sent in ontonotes_data]

	crf = sklearn_crfsuite.CRF(
		algorithm = "lbfgs",
		c1 = 1,
		c2 = 0.149853957,
		max_iterations = max_iter,
		all_possible_transitions = True,
		verbose = verbose_on
	)
	
	crf.fit(feature_lists, label_lists)
	predictions = crf.predict(feature_dicts_to_predict)
	dct = gen_NER_dict(["CARDINAL", "ORDINAL", "DATE", "NORP"], sentences_to_predict, predictions)
	
	return dct, crf

#try seeing if different hyperparamaters for each model improves
#their respsective performances, might be worth them not both being the same
def task4(ontonotes_data, sentences_to_predict):

	#print("Task 4 sentences to predict length:", len(sentences_to_predict))
	feature_lists = [gen_task4_features(sent) for sent in ontonotes_data]
	feature_dicts_to_predict = [gen_task4_features(sent) for sent in sentences_to_predict]
	#print("The first feature dict in the first feature dict list is:")
	#print(feature_lists[0][0])
	#print("The first feature dict in the first feature dict list (the ones that should be predicted) is:")
	#print(feature_dicts_to_predict[0][0])
	label_lists = [sent2labels(sent) for sent in ontonotes_data]
	crf = sklearn_crfsuite.CRF(
		algorithm = "lbfgs",
		c1 = 1,
		c2 = 0.149853957,
		max_iterations = max_iter,
		all_possible_transitions = True,
		verbose = verbose_on)
	crf.fit(feature_lists, label_lists)
	
	#REPORT STUFF GOES HERE
	
	predictions = crf.predict(feature_dicts_to_predict)
	
	dct = gen_NER_dict(["PERSON"], sentences_to_predict, predictions)
	#predictions = crf.predict(feature_dicts_to_predict)
	#print("TASK 4 PREDICTIONS:")
	#print(gen_tok_NER_pair_lists(sentences_to_predict, predictions))
	return dct, crf
	#return extract_entities_from_sequences(["PERSON"], predictions, False)

task3dict, task3model = task3(wtask3, sentences_to_predict)
task4dict, task4model = task4(wtask4, sentences_to_predict)

#combine into one dict as required
task3dict.update(task4dict)

return task3dict

num_care is: 6000
num_dont_care is: 14000
num_care is: 6000
num_dont_care is: 14000


loading training data to CRFsuite:  10%|▉         | 3241/33300 [00:02<00:25, 1179.69it/s]


KeyboardInterrupt: 

In [205]:
def report(trained_model, bestN, worstN, tagset):
	
	print("TRANSITION WEIGHTS LEARNT ON DATASET:\n")
	display( eli5.show_weights(trained_model, top=(bestN,worstN), targets = tagset, show=['transition_features']))
	print("\nBEST PERFORMING FEATURES FOR EACH NER TAG:\n")
	display( eli5.show_weights(trained_model, top=(bestN,worstN), targets = tagset, show=['targets']) )

In [212]:
result, model = task3(wtask3, sentences_to_predict)

loading training data to CRFsuite: 100%|██████████| 33300/33300 [00:36<00:00, 911.05it/s] 



Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 487607
Seconds required: 6.118

L-BFGS optimization
c1: 1.000000
c2: 0.149854
num_memories: 6
max_iterations: 125
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=4.35  loss=1060374.79 active=315351 feature_norm=1.00
Iter 2   time=4.33  loss=638279.94 active=310349 feature_norm=4.33
Iter 3   time=2.17  loss=552445.75 active=236867 feature_norm=3.87
Iter 4   time=8.98  loss=447526.29 active=152001 feature_norm=2.71
Iter 5   time=2.32  loss=384348.08 active=247396 feature_norm=3.71
Iter 6   time=2.39  loss=307577.04 active=239469 feature_norm=3.88
Iter 7   time=4.46  loss=261900.76 active=196689 feature_norm=4.75
Iter 8   time=2.39  loss=241897.59 active=201299 feature_norm=4.95
Iter 9   time=2.26  loss=219996.98 active=184624 feature_norm

Iter 120 time=2.22  loss=28722.93 active=28284 feature_norm=94.84
Iter 121 time=2.24  loss=28719.31 active=28266 feature_norm=94.89
Iter 122 time=2.18  loss=28710.56 active=28228 feature_norm=94.91
Iter 123 time=2.17  loss=28707.39 active=28181 feature_norm=94.97
Iter 124 time=2.20  loss=28698.01 active=28140 feature_norm=94.99
Iter 125 time=2.21  loss=28694.35 active=28088 feature_norm=95.04
L-BFGS terminated with the maximum number of iterations
Total seconds required for training: 294.160

Storing the model
Number of active features: 28088 (487607)
Number of active attributes: 14893 (330170)
Number of active labels: 9 (9)
Writing labels
Writing attributes
Writing feature references for transitions
Writing feature references for attributes
Seconds required: 0.012



In [213]:
result

{'CARDINAL': ['seven', 'one', 'two', 'three', 'a couple'],
 'ORDINAL': ['first'],
 'DATE': ['some days',
  'at least half-a-dozen years',
  'about a month',
  'many years ago'],
 'NORP': ['indians']}

In [109]:
task4_tagset = ["B-PERSON", "I-PERSON"]
task3_tagset = ["B-DATE", "I-DATE", "B-CARDINAL", "I-CARDINAL", "B-ORDINAL", "I-ORDINAL", "B-NORP", "I-NORP"]

report(model, 50, 20, task3_tagset)

TRANSITION WEIGHTS LEARNT ON DATASET:



From \ To,B-DATE,I-DATE,B-CARDINAL,I-CARDINAL,B-ORDINAL,I-ORDINAL,B-NORP,I-NORP
B-DATE,-5.697,3.683,-2.501,-4.576,-2.046,-0.981,-0.197,-0.882
I-DATE,-5.856,4.606,-2.085,-3.915,-1.111,-1.394,-0.992,-1.639
B-CARDINAL,-2.187,-5.853,-4.47,2.276,-1.528,-1.598,-0.033,-1.495
I-CARDINAL,-1.968,-4.013,-3.286,3.677,-0.447,-0.493,0.065,-1.26
B-ORDINAL,-1.632,-4.903,-1.916,-2.904,-1.328,4.564,0.0,0.0
I-ORDINAL,-1.33,-1.429,-0.654,-1.609,0.0,4.752,0.0,0.0
B-NORP,-3.429,-4.73,-2.789,-4.468,-2.109,-1.311,-3.257,3.647
I-NORP,-2.733,-2.606,-1.765,-2.806,-0.61,0.0,-1.081,4.844



BEST PERFORMING FEATURES FOR EACH NER TAG:



In [ ]:
print("task 3 processed size:")
print(len(task3_processed))

print("task 4 processed size:")
print(len(task4_processed))

#Think this is all that my compie can handle :)
task3_processed = random_sample(task3_processed, sample_size)
task4_processed = random_sample(task4_processed, sample_size)

print("TASK 3 SIZE:")
print(len(task3_processed))

print("TASK 4 SIZE:")
print(len(task4_processed))

#ner_dict = task3(task3_processed, sentences_to_predict)
#ner_dict.update(task4(task4_processed, sentences_to_predict))

In [243]:
str = "I HAD led this life about a month, when the man with the wooden leg began to stump about with a mop and a bucket of water, from which I inferred that preparations were making to receive Mr. Creakle and the boys"